In [ ]:
import os
import pandas as pd
import logging
from pathlib import Path
import shutil

import eikon as ek 
ek.set_app_key('dbece80752cb4d4089fbfe4a483d7a38c8f06b51')
 

In [ ]:
# Create a list of forecasters

forec=pd.read_excel(r"C:\Users\K2149424\OneDrive - King's College London\KCL\Ana\forex_survey\Data\AUD_list.xlsx")
directory = r"C:\Users\K2149424\OneDrive - King's College London\KCL\Ana\forex_survey\Data\list_forcasters"
i = 0 
for filename in os.listdir(directory):
    i += 1
    if filename.endswith(".xlsx") :
        df1 = pd.read_excel(os.path.join(directory, filename))
        forec = pd.merge(forec, df1 , how='inner', on=['Contributor'])
    else:
        continue

forec_list = forec[['RIC_y','Contributor']]
forec_list = forec_list.drop_duplicates(subset=['Contributor'])
forec_list.rename(columns={'RIC_x':'RIC'}, inplace=True)
forec_list.to_excel(r"C:\Users\K2149424\OneDrive - King's College London\KCL\Ana\forex_survey\Data\All.xlsx")

## Currency-wise forecaster file generator

In [ ]:
currency = ['AUD','EUR','GBP','JPY','NZD','CHF','NOK','SEK', 'CAD']
forec = pd.read_excel(r"C:\Users\K2149424\OneDrive - King's College London\KCL\Ana\forex_survey\Data\All.xlsx",converters={'RIC': lambda x: str(x)})
forec = forec[['RIC']]
freq = ['1M','3M','6M','1Y']


for curr in currency:
    directory = str(curr)
    parent_dir = r"C:\Users\K2149424\OneDrive - King's College London\KCL\Ana\forex_survey\Data\api_data"
    path = os.path.join(parent_dir, directory)
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)
    for index, roww in forec.iterrows():
        df1 = []
        for row in freq :
            try:
                print(str(curr)+str(row)+'P='+str(roww['RIC']))
                df = ek.get_timeseries(str(curr)+str(row)+'P='+str(roww['RIC']), fields='*', start_date='2005-01-01', end_date=None, interval='monthly')
                df.rename(columns={'VALUE': str(curr)+str(row)+'P='+str(roww['RIC'])}, inplace=True)
                df1.append(df)
                time.sleep(5)
                
            except:
                pass
        if len(df1) != 0:
            df_final =  pd.concat(df1, axis=1)  
            df_final.dropna()
            df_final.to_excel(str(curr)+"_"+str(roww['RIC'])+".xlsx") 
          
